# 📊 Análise de Dados - Cidadãos (Camada Bronze)

Este notebook analisa os dados de cidadãos gerados na camada Bronze, incluindo:
- Quantidade de endereços por usuário
- Estruturas de formatação de endereços
- Distribuição por estados e municípios
- Visualizações e estatísticas diversas

## ⚙️ Dependências

Se necessário, instale as dependências:
```bash
pip install pandas numpy matplotlib seaborn minio pyarrow
```

## 1. Configuração e Importação de Bibliotecas

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from minio import Minio
import io
import re
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

# Configuração de visualização
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (14, 8)
plt.rcParams['font.size'] = 10

# Configurações MinIO
MINIO_SERVER_URL = "ch8ai-minio.l6zv5a.easypanel.host"
MINIO_ROOT_USER = "admin"
MINIO_ROOT_PASSWORD = "1q2w3e4r"
BUCKET_NAME = "govbr"

print("✅ Bibliotecas importadas com sucesso!")

## 2. Conexão com MinIO e Carregamento dos Dados

In [ ]:
# Conectar ao MinIO
minio_client = Minio(
    MINIO_SERVER_URL,
    access_key=MINIO_ROOT_USER,
    secret_key=MINIO_ROOT_PASSWORD,
    secure=True
)

print("✅ Conectado ao MinIO")

In [ ]:
# Listar arquivos disponíveis
print("📁 Arquivos disponíveis na camada Bronze - Cidadãos:")
print("=" * 80)

objects = list(minio_client.list_objects(BUCKET_NAME, prefix="bronze/simulado/cidadaos/", recursive=True))

if not objects:
    print("⚠️  Nenhum arquivo encontrado. Execute primeiro o script de geração.")
else:
    for obj in objects:
        tamanho_mb = obj.size / 1024 / 1024
        print(f"  ✅ {obj.object_name} ({tamanho_mb:.2f} MB)")

In [ ]:
# Carregar dados
if objects:
    # Pegar o arquivo mais recente
    arquivo_mais_recente = max(objects, key=lambda x: x.last_modified)
    print(f"\n📂 Carregando: {arquivo_mais_recente.object_name}")
    
    # Ler dados
    response = minio_client.get_object(BUCKET_NAME, arquivo_mais_recente.object_name)
    df = pd.read_parquet(io.BytesIO(response.read()))
    
    print(f"✅ Dados carregados: {len(df):,} registros")
    print(f"\n📊 Primeiras linhas:")
    display(df.head(10))
    
    print(f"\n📋 Informações do DataFrame:")
    print(df.info())
else:
    print("⚠️  Nenhum arquivo para carregar.")
    df = None

## 3. Estatísticas Gerais

In [ ]:
if df is not None:
    print("=" * 80)
    print("📊 ESTATÍSTICAS GERAIS")
    print("=" * 80)
    
    total_registros = len(df)
    cidadaos_unicos = df['cpf'].nunique()
    
    print(f"\n📈 Totais:")
    print(f"  • Total de registros de endereços: {total_registros:,}")
    print(f"  • Total de cidadãos únicos: {cidadaos_unicos:,}")
    print(f"  • Média de endereços por cidadão: {total_registros/cidadaos_unicos:.2f}")
    
    # Estatísticas de endereços por cidadão
    enderecos_por_cidadao = df.groupby('cpf').size()
    
    print(f"\n🏠 Endereços por Cidadão:")
    print(f"  • Mínimo: {enderecos_por_cidadao.min()}")
    print(f"  • Máximo: {enderecos_por_cidadao.max()}")
    print(f"  • Mediana: {enderecos_por_cidadao.median():.2f}")
    print(f"  • Desvio padrão: {enderecos_por_cidadao.std():.2f}")
    
    # Distribuição
    print(f"\n📊 Distribuição de Endereços por Cidadão:")
    distribuicao = enderecos_por_cidadao.value_counts().sort_index()
    for num_enderecos, quantidade in distribuicao.items():
        percentual = (quantidade / cidadaos_unicos) * 100
        print(f"  • {num_enderecos} endereço(s): {quantidade:,} cidadãos ({percentual:.2f}%)")

## 4. Visualização: Distribuição de Endereços por Cidadão

In [ ]:
if df is not None:
    enderecos_por_cidadao = df.groupby('cpf').size()
    
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    
    # Gráfico 1: Histograma
    axes[0].hist(enderecos_por_cidadao, bins=15, edgecolor='black', alpha=0.7, color='skyblue')
    axes[0].set_xlabel('Número de Endereços por Cidadão', fontsize=12)
    axes[0].set_ylabel('Quantidade de Cidadãos', fontsize=12)
    axes[0].set_title('Distribuição de Endereços por Cidadão', fontsize=14, fontweight='bold')
    axes[0].grid(True, alpha=0.3)
    
    # Gráfico 2: Gráfico de barras
    distribuicao = enderecos_por_cidadao.value_counts().sort_index()
    axes[1].bar(distribuicao.index, distribuicao.values, color='coral', edgecolor='black', alpha=0.7)
    axes[1].set_xlabel('Número de Endereços', fontsize=12)
    axes[1].set_ylabel('Quantidade de Cidadãos', fontsize=12)
    axes[1].set_title('Quantidade de Cidadãos por Número de Endereços', fontsize=14, fontweight='bold')
    axes[1].grid(True, alpha=0.3, axis='y')
    
    plt.tight_layout()
    plt.show()
    
    # Estatísticas no gráfico
    print(f"\n📊 Estatísticas:")
    print(f"  • Média: {enderecos_por_cidadao.mean():.2f} endereços por cidadão")
    print(f"  • Mediana: {enderecos_por_cidadao.median():.2f} endereços por cidadão")

## 5. Análise de Estruturas de Endereços

In [ ]:
if df is not None:
    # Função para identificar padrões de formatação
    def identificar_padrao_endereco(endereco):
        """Identifica padrões de formatação nos endereços"""
        padroes = []
        
        # Verificar separadores
        if ',' in endereco:
            padroes.append('Com vírgulas')
        if '-' in endereco:
            padroes.append('Com hífens')
        if '/' in endereco:
            padroes.append('Com barras')
        if '|' in endereco:
            padroes.append('Com pipes')
        
        # Verificar CEP
        if 'CEP' in endereco.upper() or re.search(r'\d{5}-?\d{3}', endereco):
            padroes.append('Com CEP')
        
        # Verificar complemento
        if any(comp in endereco for comp in ['Apto', 'Apartamento', 'Casa', 'Sala', 'Loja', 'Bloco', 'Torre']):
            padroes.append('Com complemento')
        
        # Verificar maiúsculas/minúsculas
        if endereco.isupper():
            padroes.append('Tudo maiúsculo')
        elif endereco.islower():
            padroes.append('Tudo minúsculo')
        
        # Verificar abreviações
        if re.search(r'\b(R\.|Av\.|Pça\.|Tv\.|Al\.|Est\.|Rod\.)', endereco):
            padroes.append('Com abreviações')
        
        return ', '.join(padroes) if padroes else 'Padrão básico'
    
    # Aplicar análise
    df['padrao_formatacao'] = df['endereco'].apply(identificar_padrao_endereco)
    
    print("=" * 80)
    print("🏗️ ANÁLISE DE ESTRUTURAS DE ENDEREÇOS")
    print("=" * 80)
    
    # Contar padrões
    padroes_contagem = df['padrao_formatacao'].value_counts()
    
    print(f"\n📊 Padrões de Formatação Encontrados:")
    for padrao, quantidade in padroes_contagem.head(20).items():
        percentual = (quantidade / len(df)) * 100
        print(f"  • {padrao}: {quantidade:,} ({percentual:.2f}%)")
    
    # Exemplos de cada padrão
    print(f"\n📝 Exemplos de Endereços por Padrão:")
    for padrao in padroes_contagem.head(10).index:
        exemplo = df[df['padrao_formatacao'] == padrao]['endereco'].iloc[0]
        print(f"\n  {padrao}:")
        print(f"    {exemplo[:100]}..." if len(exemplo) > 100 else f"    {exemplo}")

In [ ]:
if df is not None:
    # Visualização dos padrões
    padroes_contagem = df['padrao_formatacao'].value_counts().head(15)
    
    fig, ax = plt.subplots(figsize=(14, 8))
    
    cores = plt.cm.Set3(range(len(padroes_contagem)))
    bars = ax.barh(range(len(padroes_contagem)), padroes_contagem.values, color=cores, edgecolor='black')
    
    ax.set_yticks(range(len(padroes_contagem)))
    ax.set_yticklabels(padroes_contagem.index, fontsize=9)
    ax.set_xlabel('Quantidade de Endereços', fontsize=12)
    ax.set_title('Top 15 Padrões de Formatação de Endereços', fontsize=14, fontweight='bold')
    ax.grid(True, alpha=0.3, axis='x')
    
    # Adicionar valores nas barras
    for i, (idx, val) in enumerate(padroes_contagem.items()):
        ax.text(val + max(padroes_contagem.values) * 0.01, i, f'{val:,}', 
                va='center', fontsize=9)
    
    plt.tight_layout()
    plt.show()

## 6. Análise por Estados

In [ ]:
if df is not None:
    # Extrair estado dos endereços
    def extrair_estado(endereco):
        """Extrai o estado (UF) do endereço"""
        # Padrão: /UF ou -UF no final
        match = re.search(r'[/-]([A-Z]{2})(?:\s|$|CEP|,)', endereco)
        if match:
            return match.group(1)
        return None
    
    df['estado'] = df['endereco'].apply(extrair_estado)
    
    print("=" * 80)
    print("🗺️ ANÁLISE POR ESTADOS")
    print("=" * 80)
    
    # Estatísticas por estado
    estados_contagem = df['estado'].value_counts()
    
    print(f"\n📊 Distribuição por Estado:")
    print(f"  • Total de estados encontrados: {estados_contagem.nunique()}")
    print(f"  • Endereços sem estado identificado: {df['estado'].isna().sum():,}")
    
    print(f"\n🏆 Top 10 Estados por Quantidade de Endereços:")
    for estado, quantidade in estados_contagem.head(10).items():
        percentual = (quantidade / len(df)) * 100
        print(f"  • {estado}: {quantidade:,} endereços ({percentual:.2f}%)")
    
    # Cidadãos únicos por estado
    print(f"\n👥 Cidadãos Únicos por Estado (Top 10):")
    cidadaos_por_estado = df.groupby('estado')['cpf'].nunique().sort_values(ascending=False)
    for estado, quantidade in cidadaos_por_estado.head(10).items():
        print(f"  • {estado}: {quantidade:,} cidadãos")

In [ ]:
if df is not None:
    estados_contagem = df['estado'].value_counts().head(15)
    
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    
    # Gráfico 1: Barras horizontais
    cores = plt.cm.viridis(np.linspace(0, 1, len(estados_contagem)))
    axes[0].barh(estados_contagem.index, estados_contagem.values, color=cores, edgecolor='black')
    axes[0].set_xlabel('Quantidade de Endereços', fontsize=12)
    axes[0].set_ylabel('Estado (UF)', fontsize=12)
    axes[0].set_title('Top 15 Estados por Quantidade de Endereços', fontsize=14, fontweight='bold')
    axes[0].grid(True, alpha=0.3, axis='x')
    
    # Gráfico 2: Pizza
    axes[1].pie(estados_contagem.values, labels=estados_contagem.index, autopct='%1.1f%%',
                startangle=90, colors=cores)
    axes[1].set_title('Distribuição Percentual por Estado', fontsize=14, fontweight='bold')
    
    plt.tight_layout()
    plt.show()

## 7. Análise por Municípios

In [ ]:
if df is not None:
    # Extrair município dos endereços
    def extrair_municipio(endereco):
        """Extrai o município do endereço"""
        # Tentar encontrar padrão comum: cidade antes do estado
        # Padrão: cidade/UF ou cidade - UF
        match = re.search(r'([A-Z][a-z]+(?:\s+[A-Z][a-z]+)*)\s*[/-]\s*([A-Z]{2})', endereco)
        if match:
            return match.group(1).strip()
        
        # Tentar padrão alternativo: cidade, estado
        match = re.search(r'([A-Z][a-z]+(?:\s+[A-Z][a-z]+)*),\s*([A-Z]{2})', endereco)
        if match:
            return match.group(1).strip()
        
        return None
    
    df['municipio'] = df['endereco'].apply(extrair_municipio)
    
    print("=" * 80)
    print("🏙️ ANÁLISE POR MUNICÍPIOS")
    print("=" * 80)
    
    # Estatísticas por município
    municipios_contagem = df['municipio'].value_counts()
    
    print(f"\n📊 Distribuição por Município:")
    print(f"  • Total de municípios encontrados: {municipios_contagem.nunique()}")
    print(f"  • Endereços sem município identificado: {df['municipio'].isna().sum():,}")
    
    print(f"\n🏆 Top 20 Municípios por Quantidade de Endereços:")
    for municipio, quantidade in municipios_contagem.head(20).items():
        percentual = (quantidade / len(df)) * 100
        print(f"  • {municipio}: {quantidade:,} endereços ({percentual:.2f}%)")
    
    # Cidadãos únicos por município
    print(f"\n👥 Cidadãos Únicos por Município (Top 10):")
    cidadaos_por_municipio = df.groupby('municipio')['cpf'].nunique().sort_values(ascending=False)
    for municipio, quantidade in cidadaos_por_municipio.head(10).items():
        print(f"  • {municipio}: {quantidade:,} cidadãos")

In [ ]:
if df is not None:
    municipios_contagem = df['municipio'].value_counts().head(20)
    
    fig, ax = plt.subplots(figsize=(14, 10))
    
    cores = plt.cm.plasma(np.linspace(0, 1, len(municipios_contagem)))
    bars = ax.barh(range(len(municipios_contagem)), municipios_contagem.values, 
                   color=cores, edgecolor='black')
    
    ax.set_yticks(range(len(municipios_contagem)))
    ax.set_yticklabels(municipios_contagem.index, fontsize=10)
    ax.set_xlabel('Quantidade de Endereços', fontsize=12)
    ax.set_title('Top 20 Municípios por Quantidade de Endereços', fontsize=14, fontweight='bold')
    ax.grid(True, alpha=0.3, axis='x')
    
    # Adicionar valores nas barras
    for i, (idx, val) in enumerate(municipios_contagem.items()):
        ax.text(val + max(municipios_contagem.values) * 0.01, i, f'{val:,}', 
                va='center', fontsize=9)
    
    plt.tight_layout()
    plt.show()

## 8. Análise de Tipos de Telefone

In [ ]:
if df is not None:
    print("=" * 80)
    print("📞 ANÁLISE DE TIPOS DE TELEFONE")
    print("=" * 80)
    
    tipos_telefone = df['tipo_telefone'].value_counts()
    
    print(f"\n📊 Distribuição por Tipo:")
    for tipo, quantidade in tipos_telefone.items():
        percentual = (quantidade / len(df)) * 100
        print(f"  • {tipo.capitalize()}: {quantidade:,} ({percentual:.2f}%)")
    
    # Visualização
    fig, ax = plt.subplots(figsize=(10, 6))
    
    cores = ['#FF6B6B', '#4ECDC4', '#45B7D1', '#FFA07A']
    ax.pie(tipos_telefone.values, labels=tipos_telefone.index.str.capitalize(), 
           autopct='%1.1f%%', startangle=90, colors=cores[:len(tipos_telefone)])
    ax.set_title('Distribuição de Tipos de Telefone', fontsize=14, fontweight='bold')
    
    plt.tight_layout()
    plt.show()

## 9. Análise de Emails

In [ ]:
if df is not None:
    print("=" * 80)
    print("📧 ANÁLISE DE EMAILS")
    print("=" * 80)
    
    # Extrair domínios
    df['dominio_email'] = df['email'].str.split('@').str[1]
    
    dominios_contagem = df['dominio_email'].value_counts()
    
    print(f"\n📊 Distribuição por Domínio:")
    print(f"  • Total de domínios únicos: {dominios_contagem.nunique()}")
    
    print(f"\n🏆 Top 10 Domínios:")
    for dominio, quantidade in dominios_contagem.head(10).items():
        percentual = (quantidade / len(df)) * 100
        print(f"  • {dominio}: {quantidade:,} ({percentual:.2f}%)")
    
    # Visualização
    fig, ax = plt.subplots(figsize=(12, 6))
    
    top_dominios = dominios_contagem.head(10)
    cores = plt.cm.Set2(range(len(top_dominios)))
    ax.barh(top_dominios.index, top_dominios.values, color=cores, edgecolor='black')
    ax.set_xlabel('Quantidade de Emails', fontsize=12)
    ax.set_ylabel('Domínio', fontsize=12)
    ax.set_title('Top 10 Domínios de Email', fontsize=14, fontweight='bold')
    ax.grid(True, alpha=0.3, axis='x')
    
    plt.tight_layout()
    plt.show()

## 10. Exemplos de Cidadãos com Múltiplos Endereços

In [ ]:
if df is not None:
    print("=" * 80)
    print("👤 EXEMPLOS DE CIDADÃOS COM MÚLTIPLOS ENDEREÇOS")
    print("=" * 80)
    
    # Encontrar cidadãos com mais endereços
    enderecos_por_cidadao = df.groupby('cpf').size()
    cidadaos_muitos_enderecos = enderecos_por_cidadao[enderecos_por_cidadao >= 10].index[:5]
    
    for idx, cpf in enumerate(cidadaos_muitos_enderecos, 1):
        cidadao_df = df[df['cpf'] == cpf].sort_values('numero_endereco')
        
        print(f"\n{'='*80}")
        print(f"Cidadão {idx}:")
        print(f"  CPF: {cpf}")
        print(f"  Nome: {cidadao_df['nome'].iloc[0]}")
        print(f"  Email: {cidadao_df['email'].iloc[0]}")
        print(f"  Telefone: {cidadao_df['telefone'].iloc[0]} ({cidadao_df['tipo_telefone'].iloc[0]})")
        print(f"  Total de Endereços: {len(cidadao_df)}")
        print(f"\n  Endereços:")
        
        for _, row in cidadao_df.iterrows():
            print(f"    {row['numero_endereco']}. {row['endereco']}")
            print(f"       Padrão: {row['padrao_formatacao']}")
            if pd.notna(row['estado']):
                print(f"       Estado: {row['estado']}")
            if pd.notna(row['municipio']):
                print(f"       Município: {row['municipio']}")
            print()

## 11. Análise de Variações de Formatação

In [ ]:
if df is not None:
    print("=" * 80)
    print("🎨 ANÁLISE DETALHADA DE VARIAÇÕES DE FORMATAÇÃO")
    print("=" * 80)
    
    # Contar quantos endereços únicos temos
    enderecos_unicos = df['endereco'].nunique()
    
    print(f"\n📊 Estatísticas de Variações:")
    print(f"  • Total de endereços únicos: {enderecos_unicos:,}")
    print(f"  • Total de registros: {len(df):,}")
    print(f"  • Taxa de unicidade: {(enderecos_unicos/len(df)*100):.2f}%")
    
    # Análise de características específicas
    caracteristicas = {
        'Com vírgulas': df['endereco'].str.contains(',').sum(),
        'Com hífens': df['endereco'].str.contains('-').sum(),
        'Com barras': df['endereco'].str.contains('/').sum(),
        'Com CEP': df['endereco'].str.contains('CEP', case=False, na=False).sum(),
        'Com complemento': df['endereco'].str.contains('Apto|Apartamento|Casa|Sala|Loja|Bloco|Torre', case=False, na=False).sum(),
        'Tudo maiúsculo': df['endereco'].str.isupper().sum(),
        'Tudo minúsculo': df['endereco'].str.islower().sum(),
        'Com abreviações': df['endereco'].str.contains(r'\b(R\.|Av\.|Pça\.|Tv\.|Al\.|Est\.|Rod\.)', regex=True, na=False).sum()
    }
    
    print(f"\n🔍 Características Encontradas:")
    for caracteristica, quantidade in caracteristicas.items():
        percentual = (quantidade / len(df)) * 100
        print(f"  • {caracteristica}: {quantidade:,} ({percentual:.2f}%)")
    
    # Visualização
    fig, ax = plt.subplots(figsize=(12, 8))
    
    caracteristicas_df = pd.DataFrame(list(caracteristicas.items()), columns=['Característica', 'Quantidade'])
    caracteristicas_df = caracteristicas_df.sort_values('Quantidade', ascending=True)
    
    cores = plt.cm.coolwarm(np.linspace(0, 1, len(caracteristicas_df)))
    ax.barh(caracteristicas_df['Característica'], caracteristicas_df['Quantidade'], 
            color=cores, edgecolor='black')
    ax.set_xlabel('Quantidade de Endereços', fontsize=12)
    ax.set_title('Características de Formatação nos Endereços', fontsize=14, fontweight='bold')
    ax.grid(True, alpha=0.3, axis='x')
    
    plt.tight_layout()
    plt.show()

## 12. Resumo Executivo

In [ ]:
if df is not None:
    print("=" * 80)
    print("📋 RESUMO EXECUTIVO")
    print("=" * 80)
    
    print(f"\n📊 DADOS GERAIS:")
    print(f"  • Total de registros: {len(df):,}")
    print(f"  • Total de cidadãos únicos: {df['cpf'].nunique():,}")
    print(f"  • Média de endereços por cidadão: {len(df)/df['cpf'].nunique():.2f}")
    
    print(f"\n🏠 ENDEREÇOS:")
    enderecos_por_cidadao = df.groupby('cpf').size()
    print(f"  • Mínimo: {enderecos_por_cidadao.min()}")
    print(f"  • Máximo: {enderecos_por_cidadao.max()}")
    print(f"  • Mediana: {enderecos_por_cidadao.median():.0f}")
    
    print(f"\n🗺️ LOCALIZAÇÃO:")
    print(f"  • Estados identificados: {df['estado'].nunique()}")
    print(f"  • Municípios identificados: {df['municipio'].nunique()}")
    
    print(f"\n📞 CONTATO:")
    print(f"  • Tipos de telefone: {df['tipo_telefone'].nunique()}")
    print(f"  • Domínios de email únicos: {df['dominio_email'].nunique()}")
    
    print(f"\n🎨 FORMATAÇÃO:")
    print(f"  • Padrões de formatação únicos: {df['padrao_formatacao'].nunique()}")
    print(f"  • Endereços únicos: {df['endereco'].nunique():,}")
    
    print(f"\n✅ Dados prontos para análise e transformação na camada Prata!")